<a href="https://colab.research.google.com/github/DipankarJDutta/DipankarJDutta-Biomarker-discovery-from-B-T-cells-in-FASD-GDM-with-LSTM/blob/Feature-extraction-with-Shap/LSTM_B_T_Median_optimized%20model_SHAP%20feature%20extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Fixing random seed to 007 for reproducibility
from numpy.random import seed
seed(7)
from tensorflow import set_random_seed
set_random_seed(7)

In [59]:
#Installing SHAP (SHapley Additive exPlanations)
!pip install shap

In [0]:
#Import SHAP specific essentials
from __future__ import print_function
from keras.preprocessing import sequence
from keras.layers import Embedding


In [0]:
# Loading essentials
from numpy import loadtxt
from numpy import reshape
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [0]:
# Load training data
x_train = loadtxt('B-T-median.csv', delimiter = ',')
x_test = loadtxt('B-T-median.csv', delimiter = ',')

In [63]:
# Reshape 2D training dataset into a 3D dataset with columns as features with one time-step
x_train = x_train.reshape(56, 1, 30)
x_test = x_test.reshape(56, 1, 30)

# Print dataset shape
x_train.shape
x_test.shape

(56, 1, 30)

In [0]:
#Specify binary output of good (1) and bad (0) learners, y_train and y_test, in x_train and x_test respectively. Learner type differentiated by population median.
y_train = x_train [:, :, -1]
y_test = x_test [:, :, -1]


In [0]:
# Define LSTM One-to-One Model with 3 LSTM cells (determined via hyperparameter optimization). Sigmoid activation on the output layer for binary classification.
model = Sequential()
model.add(LSTM(3, input_shape=(1,30)))
model.add(Dense(1, activation='sigmoid'))

In [0]:
# Compile the model 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [67]:
#Fit the model with all of the data as train and test samples
model.fit(x_train, y_train, batch_size = 8, epochs = 1000, shuffle = True, validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test, batch_size=8)
print('Test score:', score)
print('Test accuracy:', acc)

Train on 56 samples, validate on 56 samples
Epoch 1/1000
56/56 [==============================] - 2s 43ms/step - loss: 0.7107 - acc: 0.4464 - val_loss: 0.7086 - val_acc: 0.4464
Epoch 2/1000
56/56 [==============================] - 0s 829us/step - loss: 0.7075 - acc: 0.4464 - val_loss: 0.7059 - val_acc: 0.4464
Epoch 3/1000
56/56 [==============================] - 0s 657us/step - loss: 0.7051 - acc: 0.4464 - val_loss: 0.7033 - val_acc: 0.4464
Epoch 4/1000
56/56 [==============================] - 0s 635us/step - loss: 0.7023 - acc: 0.4464 - val_loss: 0.7012 - val_acc: 0.4464
Epoch 5/1000
56/56 [==============================] - 0s 642us/step - loss: 0.7008 - acc: 0.4464 - val_loss: 0.6989 - val_acc: 0.4464
Epoch 6/1000
56/56 [==============================] - 0s 706us/step - loss: 0.6982 - acc: 0.4464 - val_loss: 0.6972 - val_acc: 0.4464
Epoch 7/1000
56/56 [==============================] - 0s 610us/step - loss: 0.6966 - acc: 0.4464 - val_loss: 0.6954 - val_acc: 0.4464
Epoch 8/1000
56/56 

In [78]:
#Explain model with DeepExplainer from SHAP
import shap
explainer = shap.DeepExplainer(model, x_train)
shap_values = explainer.shap_values(x_test)
print(shap_values)

[array([[[-2.51292122e-03, -2.27376000e-03, -1.52087864e-04, ...,
         -3.34359809e-03, -5.05866300e-03,  3.87364493e-01]],

       [[-3.20670253e-04, -4.41124760e-04, -1.65991461e-04, ...,
         -5.58624864e-03,  2.15491548e-02,  3.85897256e-01]],

       [[ 2.30353752e-03, -1.81598092e-02, -9.47968810e-05, ...,
         -1.22885385e-02,  3.31084955e-02, -4.91970571e-01]],

       ...,

       [[ 2.33122424e-03, -8.94661679e-03,  6.48951413e-04, ...,
         -1.20892160e-02,  3.28052800e-02, -4.91291779e-01]],

       [[ 2.25851732e-03, -1.07697436e-02,  3.38616308e-04, ...,
         -1.17822110e-02,  3.17686353e-02, -4.75508703e-01]],

       [[ 2.12903704e-03, -8.17722158e-03,  4.15602448e-04, ...,
          3.27591770e-03, -7.79808845e-03,  4.48114918e-01]]])]
